#### Creating the tables on PostgreSQL

In [ ]:
import psycopg2

# Connect to PostgreSQL
conn = psycopg2.connect(
    dbname="ecommerce_db",
    user="postgres",
    password="",
    host="localhost",
    port="5432",
)

cur = conn.cursor()

#### Reset the tables

In [21]:
# List of tables to delete (if you want to delete specific tables)
tables_to_delete = ["products", "sales"]

# Loop through each table and drop if it exists
for table in tables_to_delete:
    try:
        # Drop the table if it exists
        cur.execute(f"DROP TABLE IF EXISTS {table} CASCADE;")
        print(f"Table {table} deleted successfully.")
    except Exception as e:
        print(f"Error occurred while deleting table {table}: {e}")

# Commit changes
conn.commit()

# cur.close()
# conn.close()

print("All tables deleted successfully.")

Table products deleted successfully.
Table sales deleted successfully.
All tables deleted successfully.


#### Create the two tables

In [22]:
# Create the two tables based on the after_cleaning.csv
create_products_table = """
CREATE TABLE IF NOT EXISTS products (
    product_id SERIAL PRIMARY KEY,
    date_purchased DATE,
    product_name VARCHAR(1000),
    stock VARCHAR(20),
    price NUMERIC,
    discount NUMERIC,  
    order_quantity NUMERIC
);
"""

create_sales_table = """
CREATE TABLE IF NOT EXISTS sales (
    sale_id SERIAL PRIMARY KEY,
    product_id INT REFERENCES products(product_id),
    discounted_price NUMERIC,
    total_sales_discounted NUMERIC,
    discount_amount NUMERIC,
    date_purchased DATE
);
"""

# Execute the SQL commands
cur.execute(create_products_table)
cur.execute(create_sales_table)

# Commit changes to the database
conn.commit()

#### Inserting the data from csv to the database

In [23]:
import pandas as pd
import psycopg2
from psycopg2 import sql

# Load the csv file into pandas
file_path = "./exports/after_cleaning.csv"

data = pd.read_csv(file_path)

In [24]:
data.head(5)

,price,discount,product_name,order_quantity,discounted_price,total_sales_discounted,discount_amount
0,2.03,22,0,NaN,1.5834,NaN,0.4466
1,6.48,20,0,NaN,5.1840,NaN,1.2960
2,1.80,0,0,400.0,1.8000,720.00,0.0000
3,0.88,72,0,5600.0,0.2464,1379.84,0.6336
4,12.06,40,0,NaN,7.2360,NaN,4.8240


#### Separation of data into two datafarmes

In [25]:
# Separate the product data and sales data
products_df = data[["product_name", "price", "discount", "order_quantity"]].head(500)
sales_df = data[["discounted_price", "total_sales_discounted", "discount_amount"]].head(
    500
)

#### Create product_id and date_purchased as new columns

In [26]:
from datetime import timedelta, date
import numpy as np

# Number of products to assign in the first 6 months (40% of the total products)
num_first_six_months = int(len(products_df) * 0.4)

# Generate random dates for the first 6 months (January to June)
start_date = date(2024, 1, 1)
end_date = date(2024, 6, 30)

# Generate random dates for the second half (after 6 months, i.e., July to December)
second_half_start = date(2024, 7, 1)
second_half_end = date(2024, 12, 31)


# Function to generate random dates within a given range
def random_date(start, end):
    delta = end - start
    return start + timedelta(days=np.random.randint(0, delta.days))


# Create 'product_id' and 'date_purchased' columns
products_df["product_id"] = range(1, len(products_df) + 1)
products_df["date_purchased"] = np.nan

# Assign dates to the first 40% products sold in the first 6 months
for idx in range(num_first_six_months):
    products_df.loc[idx, "date_purchased"] = random_date(start_date, end_date)

# Assign dates to the remaining 60% products sold after 6 months
for idx in range(num_first_six_months, len(products_df)):
    products_df.loc[idx, "date_purchased"] = random_date(
        second_half_start, second_half_end
    )

# Reorder columns
products_df = products_df[
    ["product_id", "date_purchased"]
    + [
        col
        for col in products_df.columns
        if col not in ["product_id", "date_purchased"]
    ]
]

# Display the updated DataFrame
products_df

C:\Users\Jerico\AppData\Local\Temp\ipykernel_16668\1890664752.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2024-03-10' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  products_df.loc[idx, "date_purchased"] = random_date(start_date, end_date)


,product_id,date_purchased,product_name,price,discount,order_quantity
0,1,2024-03-10,0,2.03,22,NaN
1,2,2024-05-30,0,6.48,20,NaN
2,3,2024-03-29,0,1.80,0,400.0
3,4,2024-01-22,0,0.88,72,5600.0
4,5,2024-03-22,0,12.06,40,NaN
...,...,...,...,...,...,...
495,496,2024-08-29,"Mini Electric Shaver,Small And Compact Efficie...",4.40,4,300.0
496,497,2024-08-12,"1pc Teeth Whitening LED Light Accelerator, Wit...",11.40,3,50.0
497,498,2024-09-12,1pc Rechargeable Automatic Foam Soap Dispenser...,20.90,0,40.0
498,499,2024-11-04,The New 1PC Rechargeable Electric Toothbrush H...,12.66,40,NaN


#### Create stock column as a category of products

In [27]:
import numpy as np

# Get the total number of products
total_products = len(products_df)

# Number of products to be marked as "available" (40% of total products)
num_available = int(total_products * 0.4)

# Randomly select 40% of the 'product_id' for "available"
available_ids = np.random.choice(
    products_df["product_id"], num_available, replace=False
)

# Create the 'stock' column with default value "unavailable"
products_df["stock"] = "unavailable"

# Set the 'stock' value to "available" for the selected product_ids
products_df.loc[products_df["product_id"].isin(available_ids), "stock"] = "available"

# Reorder columns to have 'product_id', 'date_purchased', and 'stock' as the first three columns
products_df = products_df[
    ["product_id", "date_purchased", "stock"]
    + [
        col
        for col in products_df.columns
        if col not in ["product_id", "date_purchased", "stock"]
    ]
]

# Display the updated DataFrame
products_df

,product_id,date_purchased,stock,product_name,price,discount,order_quantity
0,1,2024-03-10,unavailable,0,2.03,22,NaN
1,2,2024-05-30,unavailable,0,6.48,20,NaN
2,3,2024-03-29,unavailable,0,1.80,0,400.0
3,4,2024-01-22,available,0,0.88,72,5600.0
4,5,2024-03-22,unavailable,0,12.06,40,NaN
...,...,...,...,...,...,...,...
495,496,2024-08-29,unavailable,"Mini Electric Shaver,Small And Compact Efficie...",4.40,4,300.0
496,497,2024-08-12,available,"1pc Teeth Whitening LED Light Accelerator, Wit...",11.40,3,50.0
497,498,2024-09-12,unavailable,1pc Rechargeable Automatic Foam Soap Dispenser...,20.90,0,40.0
498,499,2024-11-04,unavailable,The New 1PC Rechargeable Electric Toothbrush H...,12.66,40,NaN


In [28]:
sales_df.loc[:, "sales_id"] = range(1, len(sales_df) + 1)
sales_df

,discounted_price,total_sales_discounted,discount_amount,sales_id
0,1.5834,NaN,0.4466,1
1,5.1840,NaN,1.2960,2
2,1.8000,720.00,0.0000,3
3,0.2464,1379.84,0.6336,4
4,7.2360,NaN,4.8240,5
...,...,...,...,...
495,4.2240,1267.20,0.1760,496
496,11.0580,552.90,0.3420,497
497,20.9000,836.00,0.0000,498
498,7.5960,NaN,5.0640,499


In [29]:
sales_df

,discounted_price,total_sales_discounted,discount_amount,sales_id
0,1.5834,NaN,0.4466,1
1,5.1840,NaN,1.2960,2
2,1.8000,720.00,0.0000,3
3,0.2464,1379.84,0.6336,4
4,7.2360,NaN,4.8240,5
...,...,...,...,...
495,4.2240,1267.20,0.1760,496
496,11.0580,552.90,0.3420,497
497,20.9000,836.00,0.0000,498
498,7.5960,NaN,5.0640,499


In [30]:
# Check the maximum value in each column
print(products_df["price"].max())
print(products_df["discount"].max())
print(products_df["order_quantity"].max())

189.99
72
10000.0


In [31]:
print(products_df.dtypes)

product_id          int64
date_purchased     object
stock              object
product_name       object
price             float64
discount            int64
order_quantity    float64
dtype: object


#### Insert on the products table

In [ ]:
import math

# Set the max rows
max_rows = 5000

# Insert data into the products table
for _, row in products_df.head(max_rows).iterrows():
    try:
        # Extract values
        product_id = row["product_id"]
        price = row["price"]
        discount = row["discount"]
        order_quantity = row["order_quantity"]
        product_name = row["product_name"]
        date_purchased = row["date_purchased"]
        stock = row["stock"]
        # Debug print
        print(
            f"Processing row {product_name}: product_id={product_id}, price={price}, discount={discount}, order_quantity={order_quantity}, date_purchased={date_purchased}, stock={stock}"
        )

        # Handle NaN values: Replace NaN in order_quantity with 0
        if isinstance(order_quantity, float) and math.isnan(order_quantity):
            order_quantity = 0

        # Validate if the numeric fields are valid
        if not (
            isinstance(price, (int, float))
            and isinstance(discount, (int, float))
            and isinstance(order_quantity, (int, float))
        ):
            raise ValueError(f"Invalid data types found for row {product_name}")

        # Check for negative values and out of range issues
        if price < 0 or discount < 0 or order_quantity < 0:
            raise ValueError(
                f"Negative values found for price, discount, or order_quantity in row {product_name}"
            )

        if price > 1000000 or discount > 100 or order_quantity > 10000:
            raise ValueError(
                f"Out-of-range values found for price, discount, or order_quantity in row {product_name}"
            )

        # Insert into products table, now using the product_id, date_purchased, and stock from the DataFrame
        cur.execute(
            """
            INSERT INTO products 
            (product_id, product_name, price, discount, order_quantity, date_purchased, stock) 
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            """,
            (
                product_id,
                product_name,
                price,
                discount,
                order_quantity,
                date_purchased,
                stock,
            ),
        )

    except Exception as e:
        print(f"Error occurred with row {product_name}: {e}")
    conn.commit()

print("Data inserted into products table!")

Processing row 0: product_id=1, price=2.03, discount=22, order_quantity=nan, date_purchased=2024-03-10, stock=unavailable
Processing row 0: product_id=2, price=6.48, discount=20, order_quantity=nan, date_purchased=2024-05-30, stock=unavailable
Processing row 0: product_id=3, price=1.8, discount=0, order_quantity=400.0, date_purchased=2024-03-29, stock=unavailable
Processing row 0: product_id=4, price=0.88, discount=72, order_quantity=5600.0, date_purchased=2024-01-22, stock=available
Processing row 0: product_id=5, price=12.06, discount=40, order_quantity=nan, date_purchased=2024-03-22, stock=unavailable
Processing row 0: product_id=6, price=2.7, discount=0, order_quantity=nan, date_purchased=2024-03-14, stock=unavailable
Processing row 0: product_id=7, price=3.5, discount=0, order_quantity=nan, date_purchased=2024-01-29, stock=unavailable
Processing row 0: product_id=8, price=4.9, discount=0, order_quantity=nan, date_purchased=2024-03-15, stock=available
Processing row 0: product_id=9

#### Insert on the sales table

In [ ]:
import math


for _, row in sales_df.head(max_rows).iterrows():
    try:
        sales_id = int(row["sales_id"])
        discounted_price = float(row["discounted_price"])
        total_sales_discounted = row["total_sales_discounted"]
        discount_amount = float(row["discount_amount"])

        # Handle NaN values: Replace NaN in total_sales_discounted with 0
        if isinstance(total_sales_discounted, float) and math.isnan(
            total_sales_discounted
        ):
            total_sales_discounted = 0
        else:
            total_sales_discounted = float(total_sales_discounted)

        # Handle NaN in other columns if necessary (e.g., discounted_price, discount_amount)
        if isinstance(discounted_price, float) and math.isnan(discounted_price):
            discounted_price = 0
        if isinstance(discount_amount, float) and math.isnan(discount_amount):
            discount_amount = 0

        # Fetch product_id and date_purchased from the products table based on sales_id
        cur.execute(
            "SELECT product_id, date_purchased FROM products WHERE product_id = %s",
            (sales_id,),
        )
        result = cur.fetchone()

        if result is None:
            raise ValueError(f"Product ID {sales_id} not found in products table")

        product_id = result[0]
        date_purchased = result[1]

        # Debug print
        print(
            f"Processing sales_id {sales_id}: product_id={product_id}, "
            f"discounted_price={discounted_price}, total_sales_discounted={total_sales_discounted}, "
            f"discount_amount={discount_amount}, date_purchased={date_purchased}"
        )

        # Validate the numeric fields
        if not (
            isinstance(discounted_price, (int, float))
            and isinstance(total_sales_discounted, (int, float))
            and isinstance(discount_amount, (int, float))
        ):
            raise ValueError(f"Invalid data types found for sales_id {sales_id}")

        # Check for negative values and out-of-range issues
        if discounted_price < 0 or total_sales_discounted < 0 or discount_amount < 0:
            raise ValueError(f"Negative values found for sales_id {sales_id}")

        # Insert into sales table with the sales_id, product_id, date_purchased, and other fields
        cur.execute(
            """
            INSERT INTO sales 
            (sale_id, product_id, discounted_price, total_sales_discounted, discount_amount, date_purchased) 
            VALUES (%s, %s, %s, %s, %s, %s)
            """,
            (
                sales_id,
                product_id,
                discounted_price,
                total_sales_discounted,
                discount_amount,
                date_purchased,
            ),
        )

    except Exception as e:
        print(f"Error occurred with sales_id {sales_id}: {e}")

    conn.commit()

# Close the cursor and connection
cur.close()
conn.close()

print("Data inserted into sales table!")

Processing sales_id 1: product_id=1, discounted_price=1.5834, total_sales_discounted=0, discount_amount=0.4465999999999999, date_purchased=2024-03-10
Processing sales_id 2: product_id=2, discounted_price=5.184000000000001, total_sales_discounted=0, discount_amount=1.2959999999999994, date_purchased=2024-05-30
Processing sales_id 3: product_id=3, discounted_price=1.8, total_sales_discounted=720.0, discount_amount=0.0, date_purchased=2024-03-29
Processing sales_id 4: product_id=4, discounted_price=0.2464, total_sales_discounted=1379.84, discount_amount=0.6335999999999999, date_purchased=2024-01-22
Processing sales_id 5: product_id=5, discounted_price=7.236, total_sales_discounted=0, discount_amount=4.824000000000001, date_purchased=2024-03-22
Processing sales_id 6: product_id=6, discounted_price=2.7, total_sales_discounted=0, discount_amount=0.0, date_purchased=2024-03-14
Processing sales_id 7: product_id=7, discounted_price=3.5, total_sales_discounted=0, discount_amount=0.0, date_purcha

In [ ]:
import psycopg2

# PostgresQL connection
conn = psycopg2.connect(
    dbname="ecommerce_db",
    user="postgres",
    password="",
    host="localhost",
    port="5432",
)

cur = conn.cursor()

# Query to find products not sold in the last 6 months
cur.execute("""
    SELECT p.product_id, p.product_name
    FROM products p
    LEFT JOIN sales s ON p.product_id = s.product_id
    WHERE s.sale_id IS NULL OR s.date_purchased < NOW() - INTERVAL '6 months';
""")
products_not_sold = cur.fetchall()
print("Products not sold in the last 6 months:")
for product in products_not_sold:
    print(product)

Products not sold in the last 6 months:
(1, '0')
(2, '0')
(3, '0')
(4, '0')
(5, '0')
(6, '0')
(7, '0')
(8, '0')
(9, '0')
(11, '0')
(12, '0')
(14, '0')
(15, '0')
(16, '0')
(17, '0')
(18, '0')
(19, '0')
(22, '0')
(24, '0')
(25, '0')
(26, '0')
(27, '0')
(28, '0')
(29, '0')
(30, '0')
(31, '0')
(32, '0')
(33, '0')
(34, '0')
(35, '0')
(37, '0')
(38, '0')
(39, '0')
(40, '0')
(41, '4pcs Makeup Sponge Beauty Egg Set - Makeup Sponge For Foundation Blending With Egg Holder, Perfectly Used For Cream, Powder, Blush, Concealer, And Liquid')
(43, 'Portable Lithium Battery Powered Hair Trimmer & Nose Hair Trimmer')
(44, 'Multifunctional Manual Food Chopper, Kitchen Vegetable Fruit Onion Garlic Nuts Herb Slicer, Hand-Powered Mixing Cutter, Green')
(46, "Smart Electric Toothbrush With 6 Modes 42000vpm & Smart Timer - Effectively Cleans Teeth & Gums, Rechargeable USB Electric Toothbrush With 4/8 Replacement Head, Electric Toothbrush For Adults, Available In 3 Colors, Great Gift Father's Day Gift")
(47, '

In [35]:
cur.execute("SELECT product_id, product_name FROM products;")
# Fetch all the results
products = cur.fetchall()

# Print each product's ID and name
print("Product ID and Product Name:")
for product in products:
    print(f"Product ID: {product[0]}, Product Name: {product[1]}")

Product ID and Product Name:
Product ID: 1, Product Name: 0
Product ID: 2, Product Name: 0
Product ID: 3, Product Name: 0
Product ID: 4, Product Name: 0
Product ID: 5, Product Name: 0
Product ID: 6, Product Name: 0
Product ID: 7, Product Name: 0
Product ID: 8, Product Name: 0
Product ID: 9, Product Name: 0
Product ID: 10, Product Name: 0
Product ID: 11, Product Name: 0
Product ID: 12, Product Name: 0
Product ID: 13, Product Name: 0
Product ID: 14, Product Name: 0
Product ID: 15, Product Name: 0
Product ID: 16, Product Name: 0
Product ID: 17, Product Name: 0
Product ID: 18, Product Name: 0
Product ID: 19, Product Name: 0
Product ID: 20, Product Name: 0
Product ID: 21, Product Name: 0
Product ID: 22, Product Name: 0
Product ID: 23, Product Name: 0
Product ID: 24, Product Name: 0
Product ID: 25, Product Name: 0
Product ID: 26, Product Name: 0
Product ID: 27, Product Name: 0
Product ID: 28, Product Name: 0
Product ID: 29, Product Name: 0
Product ID: 30, Product Name: 0
Product ID: 31, Prod

#### It gets the products that have not been sold in the last 6 months. The first few rows are 0 because the the null product name is set with a default value of 0.

#### Get the total revenu based if the product is in stock

In [36]:
import psycopg2


def get_total_revenue_by_availability():
    try:
        # Query to calculate the total revenue for available and unavailable products
        cur.execute("""
            SELECT 
                p.stock, 
                COALESCE(SUM(s.discounted_price * s.total_sales_discounted), 0) AS total_revenue
            FROM products p
            LEFT JOIN sales s ON p.product_id = s.product_id
            WHERE p.stock IN ('available', 'unavailable')  -- Filter by availability status
            GROUP BY p.stock;  -- Group by availability status
        """)

        # Fetch the results (total revenue for available and unavailable products)
        results = cur.fetchall()

        # Print the results
        for row in results:
            stock_status = row[0]
            total_revenue = row[1]
            print(f"Total revenue for '{stock_status}' products: {total_revenue:.2f}")

    except Exception as e:
        print(f"Error occurred: {e}")
    finally:
        # Close the cursor and connection
        cur.close()
        conn.close()


# Call the function to get total revenue for available and unavailable products
get_total_revenue_by_availability()

Total revenue for 'available' products: 2169856.41
Total revenue for 'unavailable' products: 3000430.00


#### Using stored procedure:

In [ ]:
# This is the stored proceudre created on the PostgreSQL database

# CREATE OR REPLACE FUNCTION public.calculate_total_revenue_by_stock(stock_status text)
# RETURNS numeric
# LANGUAGE plpgsql
# AS $function$
# DECLARE
#     total_revenue NUMERIC;
# BEGIN
#     -- Calculate the total revenue for the given stock status (available or unavailable)
#     SELECT COALESCE(SUM(s.discounted_price * s.total_sales_discounted), 0)
#     INTO total_revenue
#     FROM public.products p
#     LEFT JOIN public.sales s ON p.product_id = s.product_id
#     WHERE p.stock = stock_status;

#     RETURN total_revenue;
# END;
# $function$;


In [ ]:
import psycopg2


def get_total_revenue_by_stock(stock_status):
    try:
        # Establish a connection to the PostgreSQL database using a context manager
        with psycopg2.connect(
            dbname="ecommerce_db",
            user="postgres",
            password="",
            host="localhost",
            port="5432",
        ) as conn:
            with conn.cursor() as cur:
                cur.execute(
                    "SELECT public.calculate_total_revenue_by_stock(%s);",
                    (stock_status,),
                )

                total_revenue = cur.fetchone()[0]

                print(
                    f"Total revenue for '{stock_status}' products: {total_revenue:.2f}"
                )

    except psycopg2.DatabaseError as e:
        print(f"Database error occurred: {e}")
    except Exception as e:
        print(f"Error occurred: {e}")


# Get total revenue for 'available' products
get_total_revenue_by_stock("available")

# Get total revenue for 'unavailable' products
get_total_revenue_by_stock("unavailable")


Total revenue for 'available' products: 2169856.41
Total revenue for 'unavailable' products: 3000430.00
